# SMR to WAV Conversion (0-1000 Hz)

Converts Spike2 `.smr` recordings into filtered, normalized WAV segments that match the GUI defaults (100 kHz, 0-1000 Hz).

**Platform note:** `neo.io.CedIO` + `sonpy` are only available on x86_64. On Apple Silicon, run this on an Intel/AMD machine or use pre-converted WAVs.

In [ ]:
import os
import platform
import numpy as np
import soundfile as sf
from scipy import signal
try:
    from neo.io import CedIO
    NEO_AVAILABLE = True
except Exception as exc:
    NEO_AVAILABLE = False
    NEO_IMPORT_ERROR = exc

SRC_DIR = 'Mech_data_smr'
OUTPUT_DIR = 'Wav_data_mech'
SAMPLING_RATE = 100000  # Hz
FILTER_ORDER = 4
BANDPASS = (0, 1000)  # Hz
SEGMENTS = {
    'forward': (0.0, 2.15),
    'backward': (2.5, 4.7),
}  # seconds
os.makedirs(OUTPUT_DIR, exist_ok=True)

print('Source:', SRC_DIR)
print('Output:', OUTPUT_DIR)
print('Sampling rate:', SAMPLING_RATE, 'Hz')
print('Bandpass:', BANDPASS, 'Hz')

In [ ]:
arch = platform.machine()
if arch == 'arm64':
    print('Warning: Detected Apple Silicon (arm64). CedIO/sonpy may be unavailable here.')
if not NEO_AVAILABLE:
    print('neo/sonpy not available:', NEO_IMPORT_ERROR)
    print('Install on x86_64 or use pre-converted WAVs.')

In [ ]:
def load_smr(filepath, channel=0):
    if not NEO_AVAILABLE:
        raise ImportError(f'neo.io.CedIO unavailable: {NEO_IMPORT_ERROR}')
    reader = CedIO(filename=filepath)
    block = reader.read_block(signal_group_mode='all')
    asig = block.segments[0].analogsignals[channel]
    data = np.array(asig).squeeze().astype(float)
    sr = float(asig.sampling_rate)
    return data, sr


def butter_bandpass_filter(data, fs, band=BANDPASS, order=FILTER_ORDER):
    nyq = 0.5 * fs
    low, high = band
    sos = signal.butter(order, [low/nyq, high/nyq], btype='band', output='sos')
    return signal.sosfiltfilt(sos, data)


def normalize_audio(data):
    peak = np.max(np.abs(data))
    return data if peak == 0 else data / peak


def segment_and_save(data, sr, species, direction, stem):
    for seg_name, (start_s, end_s) in SEGMENTS.items():
        start_idx = int(start_s * sr)
        end_idx = int(end_s * sr)
        if start_idx >= len(data):
            continue
        segment = data[start_idx:min(end_idx, len(data))]
        out_dir = os.path.join(OUTPUT_DIR, species, direction or seg_name)
        os.makedirs(out_dir, exist_ok=True)
        out_path = os.path.join(out_dir, f"{stem}_{seg_name}.wav")
        sf.write(out_path, segment, int(SAMPLING_RATE))
        print(f"  -> Saved {out_path}")


def process_single_file(filepath):
    species = os.path.basename(os.path.dirname(filepath))
    stem = os.path.splitext(os.path.basename(filepath))[0]
    raw, sr_raw = load_smr(filepath)
    filtered = butter_bandpass_filter(raw, sr_raw)
    if sr_raw != SAMPLING_RATE:
        filtered = signal.resample(filtered, int(len(filtered) * SAMPLING_RATE / sr_raw))
    normalized = normalize_audio(filtered)
    segment_and_save(normalized, SAMPLING_RATE, species, direction=None, stem=stem)


def batch_process(src_dir=SRC_DIR):
    if not NEO_AVAILABLE:
        raise ImportError(f'neo.io.CedIO unavailable: {NEO_IMPORT_ERROR}')
    smr_files = []
    for root, _, files in os.walk(src_dir):
        for fname in files:
            if fname.lower().endswith('.smr'):
                smr_files.append(os.path.join(root, fname))
    print(f'Found {len(smr_files)} .smr files')
    for i, fpath in enumerate(smr_files, 1):
        print(f"[{i}/{len(smr_files)}] {os.path.basename(fpath)}")
        process_single_file(fpath)


batch_process  # keep function reachable for users

In [ ]:
# Quick smoke test on the first available file (if any)
if NEO_AVAILABLE:
    found = None
    for root, _, files in os.walk(SRC_DIR):
        for fname in files:
            if fname.lower().endswith('.smr'):
                found = os.path.join(root, fname)
                break
        if found:
            break
    if found:
        print('Testing single file:', found)
        process_single_file(found)
    else:
        print('No .smr files found in', SRC_DIR)

## How to run
1) Install dependencies (`conda env create -f environment.yml` on x86_64).
2) Place Spike2 files in `Mech_data_smr/{species}/`.
3) Run the batch cell: `batch_process()` to write WAVs into `Wav_data_mech/{species}/{segment}/`.
4) Open `Frequency_Range_Analysis.ipynb` or the GUI and analyse the generated WAVs.